https://pythonhosted.org/pyserial/

input
python -m serial.tools.list_ports -v
in anconda to view all ports available

default communication parameters E5CN-H
Communications protocol: CompoWay/F
Communications unit number: 1
Communications baud rate: 9.6 (kbit/s)
Communications data length: 7 (bit)
Communications stop bits:   2
Communications parity: Even
Send data wait time: 20 ms

In [1]:
import serial
ser = serial.Serial('COM4', 9600, bytesize=7, parity=serial.PARITY_EVEN, stopbits=2, xonxoff=0, rtscts=0, timeout=1)
ser.is_open

SerialException: could not open port 'COM4': FileNotFoundError(2, 'Het systeem kan het opgegeven bestand niet vinden.', None, 2)

In [6]:
def bcccalc(data):
    bcc=data[1]
    for i in range(2,len(data)):
        bcc=bcc ^ data[i]  
    return chr(bcc).encode('UTF-8')

def datagen(temp, controller): #enter temp as float st ##.## and controllor as int 1,2 or 3

    stx=b'\x02'
    etx=b'\x03'
    subadress=b'00'
    SID=b'0'

    controllerbit=('0'+str(controller)).encode('UTF-8') #either 01,02,03
    command=b'0102C4000000000100000' #for changing the set point of the first bank, which is what you want
    temp=int(round(temp*100)) #rounding 
    hexnmb=hex(temp)[2:].upper().encode('UTF-8')
    data=stx+controllerbit+subadress+SID+command+hexnmb+etx
    databcc=data+bcccalc(data)
    return databcc

def changet(temp, controller):
    data=datagen(temp,controller)
    ser.close()
    ser.open()
    ser.write(data)
    #print(ser.read(40))

In [8]:
import time

def ramp(Tinit,Tend,dT,totaltime):
    t00=time.clock()
    steps=int(round((Tend-Tinit)/dT))
    Trange=[Tinit + i*dT for i in range(0,steps+1)]
    print('Trange is given by '+str(Trange))
    waittime=totaltime/len(Trange) 
    print('Waiting time between each step is '+str(waittime))
    t0=time.clock()
    print('starting at time '+str(int(round(t0))))
    for T in Trange:
        tnew=time.clock()
        print('changing temp at time '+str(int(round(tnew))))
        print('difference in time before last change '+str(int(round(tnew-t0))))
        print('difference in time before last change not rounded '+str(tnew-t0))
        t0=tnew
        changet(T,2)
        changet(T,3)
        tafterchange=time.clock()
        tcorrection=tafterchange-t0
        print('time it took to change sp '+str(int(round(tafterchange-t0))))
        print('time it took to change sp not trounded '+str(tafterchange-t0))
        time.sleep(waittime-tcorrection)
    print('total time it really took '+ str(time.clock()-t00))

In [9]:
changet(36.0,3)
changet(36.0,2)

In [10]:
ramp(36.50,37.00,0.1,1200)

Trange is given by [36.5, 36.6, 36.7, 36.8, 36.9, 37.0]
Waiting time between each step is 200.0
starting at time 82519
changing temp at time 82519
difference in time before last change 0
difference in time before last change not rounded 2.4376276996918023e-05
time it took to change sp 0
time it took to change sp not trounded 0.24984486852190457
changing temp at time 82719
difference in time before last change 200
difference in time before last change not rounded 200.0041007740947
time it took to change sp 0
time it took to change sp not trounded 0.28322882183420006
changing temp at time 82919
difference in time before last change 200
difference in time before last change not rounded 199.99233515757078
time it took to change sp 0
time it took to change sp not trounded 0.2549253978941124
changing temp at time 83119
difference in time before last change 200
difference in time before last change not rounded 200.00550134090008
time it took to change sp 0
time it took to change sp not tround

In [11]:
ramp(37.00,36.50,-0.50,500)
ramp(37.00,36.50,-0.50,500)
ramp(37.00,36.50,-0.50,500)

Trange is given by [37.0, 36.5]
Waiting time between each step is 250.0
starting at time 84971
changing temp at time 84971
difference in time before last change 0
difference in time before last change not rounded 2.3093321942724288e-05
time it took to change sp 0
time it took to change sp not trounded 0.2599930976575706
changing temp at time 85221
difference in time before last change 250
difference in time before last change not rounded 249.99807341877022
time it took to change sp 0
time it took to change sp not trounded 0.2595025785412872
total time it really took 499.99349067865114
Trange is given by [37.0, 36.5]
Waiting time between each step is 250.0
starting at time 85471
changing temp at time 85471
difference in time before last change 0
difference in time before last change not rounded 2.223800402134657e-05
time it took to change sp 0
time it took to change sp not trounded 0.25984598469221964
changing temp at time 85721
difference in time before last change 250
difference in ti